In [ ]:
using Distances, LinearAlgebra, Random, Statistics, Distributions
include("KernelQuantile.jl")
include("KernelUtils.jl")
include("KernelLogistic.jl")
include("KernelMultinomial.jl")

In [ ]:
p = 50
n = 16384
covM = ones(p,p)
for i in 1:p
    for j in 1:p
        covM[i,j] = 0.5^abs(i-j)
    end
end
λs = exp10.(range(1, -5, length=30))
ms = Int.(2 .^(range(3, 12, length=10)))
nreps = 10

In [ ]:
nreps = 10
qs = [range(3,10);]
m = 512
multinomial_errors1 = zeros(nreps,length(qs))
multinomial_errors2 = zeros(nreps,length(qs))
multinomial_errors_full1 = zeros(nreps,length(qs))
multinomial_errors_full2 = zeros(nreps,length(qs))
multinomial_errors_linear = zeros(nreps,length(qs))
multinomial_times_MM1 = zeros(nreps,length(qs))
multinomial_times_newton1 = zeros(nreps,length(qs))
multinomial_times_MM2 = zeros(nreps,length(qs))
multinomial_times_newton2 = zeros(nreps,length(qs))
multinomial_times_full1 = zeros(nreps,length(qs))
multinomial_times_full2 = zeros(nreps,length(qs))
for i in 1:nreps
    d = MvNormal(zeros(p), covM)
    X = Matrix(Transpose(rand(d,n)))
    ηs1 = -4 .+ sin.(X[:,1]).+ X[:,2] .* X[:, 3]  .+ X[:,4].^3 .- abs.(X[:,5]) .+0.1*vec(mapslices(norm, X[:,6:p]; dims=2).^2)
    ηs2 = 4 .+ cos.(X[:,1]).+ X[:,2] .* X[:, 4]  .+ X[:,5].^3 .- abs.(X[:,3]) .-0.1*vec(mapslices(norm, X[:,6:p]; dims=2).^2)
    X_test = Matrix(Transpose(rand(d,Int(n/4))))
    ηs1_test = -4 .+ sin.(X_test[:,1]).+ X_test[:,2] .* X_test[:, 3]  .+ X_test[:,4].^3 .- abs.(X_test[:,5]) .+0.1*vec(mapslices(norm, X_test[:,6:p]; dims=2).^2)
    ηs2_test = 4 .+ cos.(X_test[:,1]).+ X_test[:,2] .* X_test[:, 4]  .+ X_test[:,5].^3 .- abs.(X_test[:,3]) .-0.1*vec(mapslices(norm, X_test[:,6:p]; dims=2).^2)
    row_sums_buffer = zeros(n)
    for j in 1:length(qs)
        q = qs[j]
        ηs = [ηs1 ηs2 zeros(n,q-3)] 
        ηs_test =  [ηs1_test ηs2_test zeros(Int(n/4),q-3)] 
        P = zeros(n,q)
        P_pred = zeros(Int(n/4),q)
        compute_probs_reduced!(P,ηs, row_sums_buffer)
        Y = zeros(n,q)
        for k in 1:n
            probs = P[k, :]
            d3 = Multinomial(1, probs)
            Y[k,:] .= rand(d3)
        end
        P = zeros(Int(n/4),q)
        compute_probs_reduced!(P,ηs_test, row_sums_buffer)
        Y_test = zeros(Int(n/4),q)
        for k in 1:Int(n/4)
            probs = P[k, :]
            d3 = Multinomial(1, probs)
            Y_test[k,:] .= rand(d3)
        end
        σ = 15.0
        K = rbf_kernel(X,σ)
        G = generate_nystrom_G(n,m)
        @time B1s, total_time1 = KernelMultinomial(Y,K,G,λs,ϵ=1e-4,verbose=false)
        @time B2s, total_time2 = KernelMultinomial_newton(Y,K,G,λs,ϵ=1e-4,verbose=false)
        best_metric = -Inf
        for k in 1:length(λs)
            compute_probs_reduced!(P_pred,predict_krr(X,X_test,G,B1s[k,:,:],σ), row_sums_buffer)
            best_metric = max(loglikelihood(Y_test, P_pred),best_metric)
        end
        multinomial_errors1[i,j] = best_metric
        multinomial_times_MM1[i,j] = total_time1
        multinomial_times_newton1[i,j] = total_time2
        @time Bs, total_time = KernelMultinomial_full(Y,K,G,λs,ϵ=1e-4,verbose=false)
        best_metric = -Inf
        for k in 1:length(λs)
            compute_probs!(P_pred,predict_krr(X,X_test,G,Bs[k,:,:],σ), row_sums_buffer)
            best_metric = max(loglikelihood(Y_test, P_pred),best_metric)
        end
        multinomial_errors_full1[i,j] = best_metric
        multinomial_times_full1[i,j] = total_time
        σ = 30.0
        K = rbf_kernel(X,σ)
        @time B1s, total_time1 = KernelMultinomial(Y,K,G,λs,ϵ=1e-4,verbose=false)
        @time B2s, total_time2 = KernelMultinomial_newton(Y,K,G,λs,ϵ=1e-4,verbose=false)
        best_metric = -Inf
        for k in 1:length(λs)
            compute_probs_reduced!(P_pred,predict_krr(X,X_test,G,B1s[k,:,:],σ), row_sums_buffer)
            best_metric = max(loglikelihood(Y_test, P_pred),best_metric)
        end
        multinomial_errors2[i,j] = best_metric
        multinomial_times_MM2[i,j] = total_time1
        multinomial_times_newton2[i,j] = total_time2
        @time Bs, total_time = KernelMultinomial_full(Y,K,G,λs,ϵ=1e-4,verbose=false)
        best_metric = -Inf
        for k in 1:length(λs)
            compute_probs!(P_pred,predict_krr(X,X_test,G,Bs[k,:,:],σ), row_sums_buffer)
            best_metric = max(loglikelihood(Y_test, P_pred),best_metric)
        end
        multinomial_errors_full2[i,j] = best_metric
        multinomial_times_full2[i,j] = total_time
        B_linear,_,_ = FastMultinomial_reduced([ones(size(X,1)) X],Y)
        P_pred = zeros(Int(n/4),q)
        compute_probs_reduced!(P_pred,[ones(size(X_test,1)) X_test]*B_linear, row_sums_buffer)
        multinomial_errors_linear[i,j] = loglikelihood(Y_test, P_pred)
    end
end

In [ ]:
using DelimitedFiles
writedlm("multinomial_times_MM1_q.csv",multinomial_times_MM1,",")
writedlm("multinomial_times_newton1_q.csv",multinomial_times_newton1,",")
writedlm("multinomial_times_MM2_q.csv",multinomial_times_MM2,",")
writedlm("multinomial_times_newton2_q.csv",multinomial_times_newton2,",")
writedlm("multinomial_times_full1_q.csv",multinomial_times_full1,",")
writedlm("multinomial_times_full2_q.csv",multinomial_times_full2,",")
writedlm("multinomial_errors1_q.csv",multinomial_errors1,",")
writedlm("multinomial_errors2_q.csv",multinomial_errors2,",")
writedlm("multinomial_errors_linear_q.csv",multinomial_errors_linear,",")
writedlm("multinomial_errors_full1_q.csv", multinomial_errors_full1, ",")
writedlm("multinomial_errors_full2_q.csv", multinomial_errors_full2, ",")

In [ ]:
using Plots
using Measures
using Statistics

# Example data for demonstration (replace with your actual data)
qs = 3:10
# Create a 3x1 plot layout
plot(layout = (1,2),size=(1200,600), margin=5mm)

plot!(qs, mean(multinomial_times_MM1,dims=1)[:], yerr = std(multinomial_times_MM1,dims=1)[:], label = "Quadratic MM (standard)", xlabel = "q", ylabel = "Time (seconds)", title = "sigma=15",subplot=1,color="red",markerstrokecolor=:red)
plot!(qs, mean(multinomial_times_newton1,dims=1)[:], yerr = std(multinomial_times_newton1,dims=1)[:], label = "Newton", subplot=1,color="blue",markerstrokecolor=:blue)
plot!(qs, mean(multinomial_times_full1,dims=1)[:], yerr = std(multinomial_times_full1,dims=1)[:], label = "Quadratic MM (full)", xlabel = "q", ylabel = "Time (seconds)", title = "sigma=15",subplot=1,color="green",markerstrokecolor=:green)

plot!(qs, mean(multinomial_times_MM2,dims=1)[:], yerr = std(multinomial_times_MM2,dims=1)[:], label = "Quadratic MM (standard)", xlabel = "q", ylabel = "Time (seconds)",title = "sigma=30",subplot=2,color="red",markerstrokecolor=:red)
plot!(qs, mean(multinomial_times_newton2,dims=1)[:], yerr = std(multinomial_times_newton2,dims=1)[:], label = "Newton", subplot=2,color="blue",markerstrokecolor=:blue, legend=:none)
plot!(qs, mean(multinomial_times_full2,dims=1)[:], yerr = std(multinomial_times_full2,dims=1)[:], label = "Quadratic MM (full)", xlabel = "q", ylabel = "Time (seconds)", title = "sigma=30",subplot=2,color="green",markerstrokecolor=:green)

savefig("increaseq.pdf")